In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [12]:
datestr='3-april-2020'



In [8]:
today = pd.to_datetime(datestr.replace('-', ' '))

In [18]:
CASE_URL_FORMAT = 'https://www.health.govt.nz/system/files/documents/pages/covid-19-case-details-update-%s.xlsx'

In [19]:
with pd.ExcelFile(CASE_URL_FORMAT % datestr) as excel_file:
    cases_dfs = {name: excel_file.parse(i, header=3) for i, name in enumerate(['confirmed', 'probable'])}

In [21]:
def get_case_counts(df, date_name='Report Date', date_col=0):
    return df.groupby([df.columns[date_col], 'DHB']).agg(c=('DHB', 'count')).c.rename_axis([date_name, 'DHB'])

In [22]:
cdf = pd.concat({key: get_case_counts(df) for key, df in cases_dfs.items()}, axis=1, names=['type']).loc[:today - pd.Timedelta(days=1)]

In [24]:
totals_df = cdf.sum(1).rename('cases').reset_index()

In [25]:
px.bar(totals_df, x='Report Date', y='cases', color='DHB')

In [26]:
px.bar(totals_df, color='Report Date', y='cases', x='DHB', color_continuous_scale = px.colors.sequential.algae)

In [27]:
tdf = cdf.unstack('DHB').fillna(0).cumsum().stack('DHB')





In [28]:
tdf = tdf.assign(total = tdf.sum(1)).reset_index()
tdf = tdf[tdf.total>0]


In [29]:
tdf

type,Report Date,DHB,confirmed,probable,total
0,2020-02-26,Auckland,1.0,0.0,1.0
20,2020-02-28,Auckland,1.0,0.0,1.0
32,2020-02-28,Southern,2.0,0.0,2.0
40,2020-03-02,Auckland,1.0,0.0,1.0
52,2020-03-02,Southern,2.0,0.0,2.0
...,...,...,...,...,...
555,2020-04-02,Waikato,120.0,5.0,125.0
556,2020-04-02,Wairarapa,6.0,2.0,8.0
557,2020-04-02,Waitemata,94.0,23.0,117.0
558,2020-04-02,West Coast,3.0,0.0,3.0


In [32]:
xdf = cdf.unstack('DHB').fillna(0).cumsum()
xdf = xdf.stack(['DHB', 'type'])
xdf = xdf[xdf>0].unstack('type')
sdf = xdf.sum(0, level='Report Date').assign(DHB='total')
xdf = xdf.append(sdf.set_index('DHB', append=True), sort=True)
xdf = xdf.assign(total=xdf.sum(1)).reset_index().fillna(0)

In [33]:
px.line(xdf, x='Report Date', y='total', color='DHB', hover_data=['confirmed', 'probable'], log_y=True)

In [ ]:
fig=px.bar(xdf[xdf.DHB!='total'], x='Report Date', y='total', color='DHB')
fig.data[0]

In [ ]:
DHBS = ['']

In [ ]:
df.groupby('DHB').count()

In [ ]:
cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).cumsum()

In [ ]:
px.bar(cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).reset_index(), x='Report Date', y='count', color='DHB')

In [ ]:
df2.groupby('Report Date').agg(count=('DHB', 'count'))

In [ ]:
df2.groupby('DHB').count()